# <center>Functions to Grab Datasets</center>

In [1]:
import pickle
import quandl
import pandas as pd
import html5lib

import qkey
api_key = qkey.qkey

### Webscrapes site to grab list of abbreviations of US states

In [2]:
def state_list():
    f_states= pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states')
    return f_states[0][1][1:]

### Grab all US states HPI data from quandl (saves .pickle)

In [3]:
def grab_initial_state_data():
    states = state_list()
    
    main_df = pd.DataFrame()
    
    for abbv in states:
        query = "FMAC/HPI_"+str(abbv)
        df = quandl.get(query, authtoken=api_key)
        df.drop(columns=['SA Value'], axis=1, inplace=True)      ## drops the Seasonal Adjustment column
        df.rename(columns={'NSA Value':str(abbv)}, inplace=True) ## changes the Non Seasonal Adjustment column name to the state name
        df[abbv] = (df[abbv] - df[abbv][0]) / df[abbv][0] * 100.0
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df)

    return main_df

### Grab USA HPI data from quandl (saves .pickle)

In [4]:
def HPI_Benchmark():
    
    df = quandl.get("FMAC/HPI_USA", authtoken=api_key)
    df.drop(columns=['SA Value'], axis=1, inplace=True)            ## drops the Seasonal Adjustment column
    df.rename(columns={'NSA Value':'United States'}, inplace=True) ## changes the Non Seasonal Adjustment column name to United States
    ## (new - old) / old * 100 
    df["United States"] = (df["United States"] - df["United States"][0]) / df["United States"][0] * 100.0
    return df

### Grab 30-Year Fixed-Rate Mortgage Rate (saves .pickle)
- The data is sampled at the beginning of the month, instead of the end of the month I resampled per day then averaged, then per month and averaged

In [5]:
def mortgage_30y():
    
    df = quandl.get("FMAC/30US", trim_start="1975-01-01", authtoken=api_key)
    df["Value"] = (df["Value"] - df["Value"][0]) / df["Value"][0] * 100.0
    df = df.resample('D').mean()
    df = df.resample('M').mean()
    df.columns = ['m30']
    # Drop lsast 2 rows
    df.drop(df.tail(2).index,inplace=True)
    
    return df


### Grabs US sp500 and calculates the percentage change
- The data is sampled at the beginning of the month, instead of the end of the month I resampled per day then averaged, then per month and averaged

In [6]:
def sp500_data():
    
    df = quandl.get("MULTPL/SP500_INFLADJ_MONTH", trim_start="1975-01-01", authtoken=api_key)
    df.rename({"Value": "sp500"}, axis='columns', inplace=True)
    df["sp500"] = (df["sp500"] - df["sp500"][0]) / df["sp500"][0] * 100.0
    df = df['sp500']
    df = df.resample('1D').mean()
    df = df.resample('M').mean()
    df.drop(df.tail(2).index,inplace=True)

    return df

### Grabs US GDP Data and calculates the percentage change
- The data is sampled quarterly. So I filled in the NAN with a forward fill method so I can see all months. I thought about just dropping the NAN rows but thought this would be better. Still need to do research.

In [7]:
def gdp_data():
    
    df = quandl.get("FRED/GDP", trim_start="1975-01-01", authtoken=api_key)
    df.rename(columns={'Value':'GDP'}, inplace=True)
    df.fillna(method='ffill', inplace=True)
    
    df = df.resample('M').mean()
    df.fillna(method='ffill', inplace=True)
    df = (df["GDP"] - df["GDP"][0]) / df["GDP"][0] * 100.00
    
    return df

### Grabs US Unemployment Rate and calculates the percentage change
- The data is sampled at the beginning of the month, instead of the end of the month I resampled per day then averaged, then per month and averaged

In [8]:
def us_unemployment():
    
    df = quandl.get("FRED/UNRATE", trim_start="1975-01-01", authtoken=api_key)
    df.columns = ["Unemployment Rate"]
    df["Unemployment Rate"] = (df["Unemployment Rate"] - df["Unemployment Rate"][0]) / df["Unemployment Rate"][0] * 100.0
    df = df.resample('1D').mean()
    df = df.resample('M').mean()
    df.drop(df.tail(2).index,inplace=True)

    
    return df

### Collects Data Frames and Joins Them Together

In [9]:
HPI_data = grab_initial_state_data()
HPI_benchmark = HPI_Benchmark()

m30 = mortgage_30y()
sp500 = sp500_data()
gdp = gdp_data()
unemployment = us_unemployment()

HPI = HPI_data.join([HPI_benchmark, m30, unemployment,gdp, sp500])
print(HPI)

                    AL          AK          AZ          AR           CA  \
Date                                                                      
1975-01-31    0.000000    0.000000    0.000000    0.000000     0.000000   
1975-02-28    0.537283    1.457417    1.714012    0.764938     0.383789   
1975-03-31    1.167742    2.950771    3.322759    1.401166     1.633714   
1975-04-30    1.938065    4.506952    4.556821    1.906278     3.646157   
1975-05-31    2.619518    6.206200    5.161457    2.432022     5.280673   
1975-06-30    2.873492    8.030980    4.921589    3.031004     5.944999   
1975-07-31    2.558203    9.872755    3.789761    3.542885     6.663914   
1975-08-31    1.861206   11.581457    2.076248    3.768737     8.053338   
1975-09-30    1.155799   13.083390    0.340052    3.613581     9.587789   
1975-10-31    0.844074   14.428310   -0.566710    3.310349    10.960001   
1975-11-30    1.144853   15.604692   -0.041380    3.293218    11.909927   
1975-12-31    2.090877   

### Drops Rows (dates) w/ Empty Values and Saves to .pickle

In [10]:
HPI.dropna(inplace=True)
HPI.to_pickle('HPI.pickle')